# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Toledo, Ohio, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)
    
    plt.show()

    return mplleaflet.display()

#leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [4]:
# Libraries
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np
import datetime as dt       
from matplotlib import dates, pyplot

In [5]:
def explore():
    df = pd.read_csv("fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    minDate = min(df['Date'])
    maxDate = max(df['Date'])
    df['MD'] = df['Date'].apply(lambda x: x.strftime('%m-%d'))
    dfSummary = df.groupby('MD')['Data_Value'].agg([np.min, np.max])
    return(minDate, maxDate,df, dfSummary)


In [24]:
def process():
    df = explore()[2]
    # Filter out leap days    
    df = df.loc[df["MD"] != "02-29"]
    # Filter out 2015 data
    df1 = df[(df['Date'] > dt.date(2004, 12, 31)) & (df['Date'] < dt.date(2015, 1, 1))]    
    df2 = df[(df['Date'] > dt.date(2014, 12, 31)) & (df['Date'] < dt.date(2016, 1, 1))] 
    # Create minimum and maximum data
    df1 = df1.groupby('MD')['Data_Value'].agg([np.min, np.max])
    df2 = df2.groupby('MD')['Data_Value'].agg([np.min, np.max])   
    # Merge the data sets
    df1 = df1.reset_index()
    df2 = df2.reset_index()
    df1 = df1.set_index("MD")
    df2 = df2.set_index("MD")
    df3 = pd.merge(df1, df2, how = 'inner', left_index = True, right_index = True)
    # Create record low/high data set
    dfLow = df3.loc[df3['amin_x'] > df3['amin_y']]
    dfHigh = df3.loc[df3['amax_x'] < df3['amax_y']]
    df1 = df1.reset_index()
    dfLow = dfLow.reset_index()
    dfHigh = dfHigh.reset_index()
    return(df1, dfLow, dfHigh)
df, low, high = process()
print(df.info())
#print(low.info())
#print(high)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 3 columns):
MD      365 non-null object
amin    365 non-null int64
amax    365 non-null int64
dtypes: int64(2), object(1)
memory usage: 8.6+ KB
None


In [22]:
# Prepare Data for Line Plots
xAxis = df['MD']
yLow = df['amin']
yHigh = df['amax']
# Prepare data for scatterplot
low = low.loc[:,['MD','amin_y']]
high = high.loc[:,['MD','amax_y']]
# Render line plots
fig, axs = plt.subplots()
axs.plot(xAxis, yLow)


ValueError: could not convert string to float: '12-31'